### 可视化 pymatgen Structure 对象

In [1]:
# reference: https://gist.github.com/lan496/3f60b6474750a6fd2b4237e820fbfea4
import nglview
from itertools import product
import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice
from pymatgen.core.sites import PeriodicSite


# 在 GitHub repo 中无法显示
def plot3d(
    structure: Structure,
    spacefill=True,
    show_axes=True,
):

    eps = 1e-8
    sites = []
    for site in structure:
        site: PeriodicSite
        species = site.species
        frac_coords = np.remainder(site.frac_coords, 1)
        for jimage in product([0, 1 - eps], repeat=3):
            new_frac_coords = frac_coords + np.array(jimage)
            if np.all(new_frac_coords < 1 + eps):
                new_site = PeriodicSite(
                    species=species,
                    coords=new_frac_coords,
                    lattice=structure.lattice,
                )
                sites.append(new_site)
    structure_display = Structure.from_sites(sites)

    view = nglview.show_pymatgen(structure_display)
    view.add_unitcell()

    if spacefill:
        view.add_spacefill(
            radius_type="vdw",
            radius=0.5,
            color_scheme="element",
        )
        view.remove_ball_and_stick()
    else:
        view.add_ball_and_stick()

    if show_axes:
        view.shape.add_arrow([-4, -4, -4], [0, -4, -4], [1, 0, 0], 0.5, "x-axis")
        view.shape.add_arrow([-4, -4, -4], [-4, 0, -4], [0, 1, 0], 0.5, "y-axis")
        view.shape.add_arrow([-4, -4, -4], [-4, -4, 0], [0, 0, 1], 0.5, "z-axis")

    view.camera = "perspective"
    return view

In [2]:
# rutile structure taken from mp-2657
a = 4.653
c = 2.969
x_4f = 0.3046

lattice = Lattice.from_parameters(a, a, c, 90, 90, 90)
species = ["Ti", "Ti", "O", "O", "O", "O"]
frac_coords = np.array([
    [0, 0, 0],                      # Ti(2a)
    [0.5, 0.5, 0.5],                # Ti(2a)
    [x_4f, x_4f, 0],                # O(4f)
    [1 - x_4f, 1 - x_4f, 0],        # O(4f)
    [0.5 - x_4f, 0.5 + x_4f, 0.5],  # O(4f)
    [0.5 + x_4f, 0.5 - x_4f, 0.5],  # O(4f)
])
structure = Structure(lattice, species, frac_coords)

plot3d(structure, spacefill=True)

NGLWidget()

![TiO2](assets/structure_visualize1.png)

In [3]:
structure = Structure.from_file("Nb5Si3_alpha.vasp")

plot3d(structure, spacefill=True)

NGLWidget()

![Nb5Si3_alpha](assets/structure_visualize2.png)